In [248]:
import pathlib
import pandas as pd
from tqdm import tqdm

In [250]:
for i in tqdm(range(1000000000)):
    a = 'b'

  1%|▏         | 13472818/1000000000 [00:04<05:01, 3272040.52it/s]

KeyboardInterrupt: 

  1%|▏         | 13472818/1000000000 [00:20<05:01, 3272040.52it/s]

In [84]:
# Load the raw data

data_dir = pathlib.Path.cwd().parent / 'data_raw'
bea_data_dir = data_dir / 'BEA_Industry_Factors' / 'county_level'
[f.name for f in bea_data_dir.iterdir()]

['.DS_Store',
 'industriesPerCountyGA-2016.csv',
 'industriesPerCountyGA-2017.csv',
 'industriesPerCountyGA-2015.csv',
 'industriesPerCountyGA-2000.csv',
 'industriesPerCountyGA-2014.csv',
 'industriesPerCountyGA-2007.csv',
 'industriesPerCountyGA-2013.csv',
 'industriesPerCountyGA-2012.csv']

In [93]:
# Load the 2012 data
df = pd.read_csv(bea_data_dir/'industriesPerCountyGA-2012.csv').drop("Unnamed: 11", axis=1)
df['NAICS_Sector'] = df.NAICS2012.apply(lambda v: v[:2]).apply(int)


df.head()

,GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,county,NAICS_Sector
0,0500000US13001,"Appling County, Georgia",1,2012,00,Total for all sectors,380,5369,220875,13,1,0
1,0500000US13001,"Appling County, Georgia",1,2012,11,"Agriculture, forestry, fishing and hunting",12,84,4034,13,1,11
2,0500000US13001,"Appling County, Georgia",1,2012,113,Forestry and logging,11,61,2256,13,1,11
3,0500000US13001,"Appling County, Georgia",1,2012,1133,Logging,11,61,2256,13,1,11
4,0500000US13001,"Appling County, Georgia",1,2012,11331,Logging,11,61,2256,13,1,11


In [86]:
[_ for _ in df.NAICS2012.unique() if _[:2] == '11']

['11',
 '113',
 '1133',
 '11331',
 '113310',
 '115',
 '1151',
 '11511',
 '115111',
 '115114',
 '1153',
 '11531',
 '115310',
 '115112',
 '1132',
 '11321',
 '113210',
 '1131',
 '11311',
 '113110',
 '115116',
 '11411',
 '114',
 '1141',
 '114112',
 '1142',
 '11421',
 '114210',
 '1152',
 '11521',
 '115210',
 '114119',
 '114111',
 '115115',
 '115113']

### Process FIPS Code

FIPS is the federal/census unique ID for each geographic area.  States have 2 digives and counties have 5

In [126]:
# Process FIPS code
df['fips'] = df.GEO_ID.apply(lambda GID: GID.split('US')[1])
# # Remove the state_level data
# df_state = df[df['fips'].str.len() == 2]
# df_county = df[df['fips'].str.len() != 2]

def county_level(df):
    return df[df['fips'].str.len() == 5]

def state_level(df):
    return df[df['fips'].str.len() == 5]

### Group data by NAICS Sector

NAICS is the North American Industry Classification System. The coarsest level of classification is the *Sector*.

The organization of NAICS is as follows:  <-- from [this page](https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html) on census.gov
- Sector: 2-digit code
    - Subsector: 3-digit code
        - Industry Group: 4-digit code
            - NAICS Industry: 5-digit code
                - National Industry: 6-digit code

Start by grouping the data by sector:

In [88]:
def naics_level(df, naics_level):
    return df[df['NAICS2012'].str.len() == naics_level]

In [89]:
df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df_naics_2 = naics_level(df, 2).reset_index(drop=True)
df_naics_3 = naics_level(df, 3).reset_index(drop=True)
df_naics_4 = naics_level(df, 4).reset_index(drop=True)
df_naics_5 = naics_level(df, 5).reset_index(drop=True)
df_naics_6 = naics_level(df, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df_naics_2 = df_naics_2[df_naics_2.NAICS2012 != '00']

In [90]:


# Now extract all the top-level sectors
# mask = (df['NAICS2012'].str.len() == 2) | (df['NAICS2012'].str.contains('-'))
# df[mask]
# df.query('fips==13001 & NAICS_Sector==32')

In [91]:
# county_level(df_naics_2)

In [92]:
df_naics_5[df_naics_5.fips=='13001']

,GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,county,fips,NAICS_Sector
0,0500000US13001,"Appling County, Georgia",1,2012,11331,Logging,11,61,2256,13,1,13001,11
1,0500000US13001,"Appling County, Georgia",1,2012,11511,Support activities for crop production,1,0,0,13,1,13001,11
2,0500000US13001,"Appling County, Georgia",1,2012,22111,Electric power generation,1,0,0,13,1,13001,22
3,0500000US13001,"Appling County, Georgia",1,2012,22112,"Electric power transmission, control, and dist...",6,0,0,13,1,13001,22
4,0500000US13001,"Appling County, Georgia",1,2012,23611,Residential building construction,3,0,0,13,1,13001,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,0500000US13001,"Appling County, Georgia",1,2012,81299,All other personal services,1,0,0,13,1,13001,81
151,0500000US13001,"Appling County, Georgia",1,2012,81311,Religious organizations,23,94,1669,13,1,13001,81
152,0500000US13001,"Appling County, Georgia",1,2012,81331,Social advocacy organizations,1,0,0,13,1,13001,81
153,0500000US13001,"Appling County, Georgia",1,2012,81341,Civic and social organizations,1,0,0,13,1,13001,81


---

In [54]:
# # Load NAICS 2007--> 2012 concordance
# naics_keys = pd.read_csv(data_dir / '2007_to_2012_NAICS.csv', 
#                          header=4, 
#                          usecols=['2007 NAICS Code','2007 NAICS Title','2012 NAICS Code','2012 NAICS Title'])

In [55]:
# naics_keys

In [56]:
# # Load USEEIO --> NAICS Concordance
# useeio_keys = pd.read_csv(data_dir / 'USEEIO-NAICS-Concordance_2012.csv', header=5).loc[0:958,:]

---
### Join USEEIO and BEA data

In [194]:
# Load USEEIO Indicators matrix

indicator_list = ['ACID','ENRG','ETOX','EUTR','FOOD','GCC','HAPS','HAZW','HC','HNC','HRSP','HTOX','JOBS','LAND','METL','MINE','MSW','NREN','OZON','PEST','REN','SMOG','VADD','WATR']

indicators = pd.read_csv(data_dir / 'USEEIO_Industry_Indicators' / 'indicators.csv')

# Filter out 'rous' duplicates
indicators = indicators[indicators.industry_region == 'us-ga']

# REMOVE Construction industry due to incompatible subsets
indicators = indicators[~indicators['relavant_naics_codes'].str.contains('\*')]

# Reset the index
indicators = indicators.reset_index(drop=True)

# Convert column 'relavant_naics_codes' fmor strinm to list of integers
conv_str_list = lambda x: [int(_) for _ in x.strip('][').split(', ')]
indicators.loc[:, 'relavant_naics_codes'] = indicators.loc[:, 'relavant_naics_codes'].apply(conv_str_list)

indicators.head()

,industry_code,industry_detail,industry_region,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,...,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR,Related 2012 NAICS Codes,relavant_naics_codes
0,1111a0,"fresh soybeans, canola, flaxseeds, and other o...",us-ga,0.006991,0.000000,27.141609,0.008084,0.000021,0.549898,0.000099,...,0.000083,0.0,0.000005,0.000380,0.000000,0.048809,0.471064,0.009608,11111-2,"[11111, 11112]"
1,1111b0,"fresh wheat, corn, rice, and other grains",us-ga,0.023198,0.000000,9.711579,0.033350,0.000130,1.703382,0.000131,...,0.000506,0.0,0.000006,0.000570,0.000000,0.059359,-0.002548,0.461626,"11113-6, 11119","[11113, 11114, 11115, 11116, 11119]"
2,111200,"fresh vegetables, melons, and potatoes",us-ga,0.005637,0.002340,22.179052,0.005695,0.001000,0.354328,0.000349,...,0.003883,0.0,0.000012,0.000951,0.002340,0.017317,0.604597,0.451799,1112,[1112]
3,111300,fresh fruits and tree nuts,us-ga,0.003150,0.000000,13.995239,0.000306,0.001330,0.235407,0.000091,...,0.005168,0.0,0.000006,0.000209,0.000000,0.016048,0.605345,0.403344,1113,[1113]
4,111400,"greenhouse crops, mushrooms, nurseries, and fl...",us-ga,0.000780,0.000004,0.006251,0.000097,0.001812,0.657274,0.000096,...,0.007078,0.0,0.000005,0.000046,0.000004,0.032099,0.534020,0.056251,1114,[1114]


In [118]:
for naic in indicators.loc[10, 'relavant_naics_codes']:
#     print(naic)

    test = df[df.NAICS2012 == str(naic)]

test

,GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,county,NAICS_Sector
2,0500000US13001,"Appling County, Georgia",1,2012,113,Forestry and logging,11,61,2256,13,1,11
496,0500000US13003,"Atkinson County, Georgia",3,2012,113,Forestry and logging,1,0,0,13,3,11
702,0500000US13005,"Bacon County, Georgia",5,2012,113,Forestry and logging,8,58,1879,13,5,11
1171,0500000US13009,"Baldwin County, Georgia",9,2012,113,Forestry and logging,2,0,0,13,9,11
1860,0500000US13011,"Banks County, Georgia",11,2012,113,Forestry and logging,3,0,617,13,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...
88665,0500000US13309,"Wheeler County, Georgia",309,2012,113,Forestry and logging,20,235,8219,13,309,11
88812,0500000US13311,"White County, Georgia",311,2012,113,Forestry and logging,4,0,394,13,311,11
90486,0500000US13315,"Wilcox County, Georgia",315,2012,113,Forestry and logging,3,18,517,13,315,11
90679,0500000US13317,"Wilkes County, Georgia",317,2012,113,Forestry and logging,5,52,1982,13,317,11


In [123]:
available_naics = []

for idx, row in indicators.iterrows():
    for naic in row.relavant_naics_codes:
        if str(naic) in df.NAICS2012.unique():
            available_naics.append(naic)

In [131]:
# iterate through the counties
for fips in county_level(df).fips.unique():
    # Get county data
    
    df_county = df[df.fips==fips]

In [132]:
df_county

,GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN,state,county,NAICS_Sector,fips
91332,0500000US13321,"Worth County, Georgia",321,2012,114210,Hunting and trapping,2,0,0,13,321,11,13321
91351,0500000US13321,"Worth County, Georgia",321,2012,00,Total for all sectors,275,2500,69847,13,321,0,13321
91352,0500000US13321,"Worth County, Georgia",321,2012,11,"Agriculture, forestry, fishing and hunting",5,110,3107,13,321,11,13321
91353,0500000US13321,"Worth County, Georgia",321,2012,114,"Fishing, hunting and trapping",2,0,0,13,321,11,13321
91354,0500000US13321,"Worth County, Georgia",321,2012,1142,Hunting and trapping,2,0,0,13,321,11,13321
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91764,0500000US13321,"Worth County, Georgia",321,2012,8134,Civic and social organizations,1,0,0,13,321,81,13321
91765,0500000US13321,"Worth County, Georgia",321,2012,81341,Civic and social organizations,1,0,0,13,321,81,13321
91766,0500000US13321,"Worth County, Georgia",321,2012,813410,Civic and social organizations,1,0,0,13,321,81,13321
91767,0500000US13321,"Worth County, Georgia",321,2012,8139,"Business, professional, labor, political, and ...",2,0,0,13,321,81,13321


In [251]:
entries = []

unique_fips_list = county_level(df).fips.unique()
n_unique_fips = len(unique_fips_list)

# iterate through the counties
for fips in unique_fips_list:
    # Get county data
    
    df_county = df[df.fips==fips]


    available_naics = []
    available_industry_codes = []

    for idx, row in indicators.iterrows():
        for naic in row.relavant_naics_codes:
            if str(naic) in df_county.NAICS2012.unique():
                available_naics.append(naic)
                if row.industry_code not in available_industry_codes:
                    available_industry_codes.append(row.industry_code)
                    

    for code in available_industry_codes:
        df_ind = indicators.set_index('industry_code').loc[code]

        # Make the new row
        payann_total = 0
        for naic in df_ind.relavant_naics_codes:
            # pull county data
            test = df_county[df_county.NAICS2012 == str(naic)]
            try:
                payann_total += test.PAYANN.values[0]
            except IndexError:
                continue


        new_entry = {
            'fips': df_county.iloc[0].fips,
            'county': df_county.iloc[0].COUNTY,
            'name': df_county.iloc[0].GEO_TTL,
            'industry_code': df_ind.name,
            'industry_detail': df_ind.industry_detail,
            'relevant_naics': df_ind.relavant_naics_codes,
            'year': df_county.iloc[0].YEAR,
            'payann_total': payann_total,
        }

        for indicator_name, indicator_rate in df_ind[indicator_list].iteritems():
            new_entry.update({
                indicator_name: indicator_rate * payann_total
            })

        entries.append(new_entry)


100%|██████████| 159/159 [01:20<00:00,  1.97it/s]


In [236]:
# available_naics

code = available_industry_codes[1]

# row

In [237]:
# indicator_list

In [244]:
# df_ind = indicators.set_index('industry_code').loc[code]
# # df_ind[indicator_list]
# df_ind

In [241]:
entries = []

for code in available_industry_codes:
    df_ind = indicators.set_index('industry_code').loc[code]

    # Make the new row
    payann_total = 0
    for naic in df_ind.relavant_naics_codes:
        # pull county data
        test = df_county[df_county.NAICS2012 == str(naic)]
        try:
            payann_total += test.PAYANN.values[0]
        except IndexError:
            continue

        
    new_entry = {
        'fips': df_county.iloc[0].fips,
        'county': df_county.iloc[0].COUNTY,
        'name': df_county.iloc[0].GEO_TTL,
        'industry_code': df_ind.name,
        'industry_detail': df_ind.industry_detail,
        'relevant_naics': df_ind.relavant_naics_codes,
        'year': df_county.iloc[0].YEAR,
        'payann_total': payann_total,
    }

    for indicator_name, indicator_rate in df_ind[indicator_list].iteritems():
        new_entry.update({
            indicator_name: indicator_rate * payann_total
        })
        
    entries.append(new_entry)

In [252]:
results = pd.DataFrame(data=entries)

In [253]:
results.shape

(13119, 32)

In [255]:
results.head()

,fips,county,name,industry_code,industry_detail,relevant_naics,year,payann_total,ACID,ENRG,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,13001,1,"Appling County, Georgia",113000,timber and raw forest products,[113],2012,2256,1.974009,0.369628,...,0.000175,0.0,11.856864,0.0,0.000357,0.04395,0.369628,57.543537,1340.647601,192.528636
1,13001,1,"Appling County, Georgia",115000,agriculture and forestry support,[115],2012,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
2,13001,1,"Appling County, Georgia",221100,electricity,[2211],2012,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,13001,1,"Appling County, Georgia",321100,lumber and treated lumber,[3211],2012,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
4,13001,1,"Appling County, Georgia",321910,"wooden windows, door, and flooring",[32191],2012,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [259]:
# Clean data dir
clean_data_dir = pathlib.Path.cwd().parent / 'data_clean'
results.to_csv(clean_data_dir / 'indicators_per-industry_per-county.csv')

In [258]:
clean_data_dir

PosixPath('/Users/merrelbook/Projects/HealthDataVizGA/DataVisualAnalytics_Industries-cancer/data_clean')

In [107]:
# for idx in range(len(indicators)):
#     test_code = indicators.loc[idx, 'relavant_naics_codes'][0]
#     print(f"{test_code} - {str(test_code) in df['NAICS2012'].tolist()}")

In [109]:
# "113" in df['NAICS2012'].tolist()

In [108]:
# df.query('NAICS_Sector == 11')

# df_naics_3.NAICS2012.unique()
# df.query('NAICS_Sector==11').NAICS2012_TTL.unique()

In [110]:
# df.query('NAICS_Sector == 11').NAICS2012.unique()

In [103]:
# df.NAICS2012.apply(int)